In [1]:
import pandas as pd

In [4]:
files = ['cmg', 'fpt', 'elc']
for file in files:
    df = pd.read_csv(file + '.csv');

    # Remove other column
    df = df[['Date', 'Price']]

    df['Price'] = pd.to_numeric(df['Price'].str.replace(',', ''), errors='coerce')
    
    # Convert 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Set 'Date' as the index
    df.set_index('Date', inplace=True)

    # Create a complete date range including weekends
    full_date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')

    # Reindex the DataFrame to include all dates in the range
    df = df.reindex(full_date_range)

    # Function to calculate the mean of 3 days before and 3 days after
    def mean_fill(df):
        for col in df.columns:
            for i in range(len(df)):
                if pd.isna(df.iloc[i][col]):
                    # Collect the previous 5 non-null values
                    prev_values = df[col].iloc[max(0, i - 5):i].dropna()
                    if len(prev_values) >= 1:  # Ensure there is at least one previous value to calculate the mean
                        df.iloc[i][col] = int(prev_values.mean())
        return df

    # Apply the mean fill function
    df = mean_fill(df)

    # Reset index to move 'Date' back to a column
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)
    df.sort_index(inplace=True, ascending=True)

    # Export DataFrame to CSV
    df.to_csv(file + '_data_final' + '.csv', index=False)
    
